# To do:
- test similarity scikit image (Niko)
- add scope to antspy registration (Aicha) DONE!
   Also, moved the concatentation and sim_check of the registered files to the Antspy func
- change channel 2 shifted assignment to shift 'warpmoveout' (Aicha) DONE!

##Testing:
- stage 2: T4, T5
    - sessions: 220219, 220221
- stage 1: T4, T5 
    - sessions: 211011, 220124

Define your variables

In [7]:
ants_method = ['Rigid']
path_to_data = '/Users/aichah/Desktop/TLI/data'
save_path = '/Users/aichah/Desktop/TLI/data'
group_by = '210218'
ch_names = ['GFP', 'red']
save_name = '210218_LP40_P36'
csv_name = '_220209_SyNRA_antspy.csv'

Load packages and define functions

In [8]:
import ants
from skimage import io, filters
from tqdm import tqdm
import numpy as np
import os
from timeit import default_timer as timer
import tifffile as tif
from sklearn import metrics
import csv

In [9]:
import utils.datautils as datautils

In [16]:
def antspy_drift_corr(img_4D_r, img_4D_g, ch_names, save_path, save_name, name_shifts, ref_t=0, drift_corr='Rigid'):
    """
    This function takes the folder containing all the files to each channel as an input and drift corrects them.
    It uses the files from ch1 to do the drift correction and applies the same correction to ch2.
    By default antspy uses the SyN drift correction algorithm.
    
    path_to_data_ch1: Path to files for ch1 
    path_to_data_ch2: Path to files for ch2
    savepath: Path to folder where the final files should be saved in
    name_ch1: Name for the final file for ch1
    name_ch2: Name for the final file for ch2
    name_shifts: Name for the file containing the shifts for each volume
    """

    shifts = []

    scope = np.arange(ref_t,-1,-1)
    scope = np.concatenate((scope, np.arange(ref_t-1,len(img_4D_r)-1)))

    for i in tqdm(scope):
        if i == ref_t:
            io.imsave(save_path+ch_names[0]+'_'+save_name+str(f"{i+1:03d}")+'.tif', img_4D_g[i])
            io.imsave(save_path+ch_names[1]+'_'+save_name+str(f"{i+1:03d}")+'.tif', img_4D_r[i])
            last = ants.from_numpy(np.float32(filters.median(img_4D_r[-1])))
        else:        
            moving_ch1 = ants.from_numpy(np.float32(img_4D_g[i]))
            moving_ch2 = ants.from_numpy(np.float32(img_4D_r[i]))
            
            shift = ants.registration(fixed=last, moving=moving_ch2, type_of_transform=drift_corr)
            
            vol_shifted_ch1 = ants.apply_transforms(fixed=last, moving=moving_ch1, transformlist=shift['fwdtransforms'])
            vol_shifted_ch2 = shift['warpedmovout']
            last = shift['warpedmovout'].copy()

            vol_shifted_ch1 = (vol_shifted_ch1.numpy()).astype('uint16')
            vol_shifted_ch2 = (vol_shifted_ch2.numpy()).astype('uint16')

            io.imsave(save_path+ch_names[0]+'_'+save_name+str(f"{i+1:03d}")+'.tif', vol_shifted_ch1)
            io.imsave(save_path+ch_names[1]+'_'+save_name+str(f"{i+1:03d}")+'.tif', vol_shifted_ch2)

            shifts.append(shift['fwdtransforms'])
                    
            del vol_shifted_ch1, vol_shifted_ch2, shift

    np.savetxt(X=shifts, fname=save_path+name_shifts, delimiter=', ', fmt='% s')
    return

Running the pipeline

In [4]:
# reading list of 3D images into dictionary of channels as indicated in ch_names, and applying median filter on the last ch
image_4D = datautils.read_files(path_to_data, group_by ,compile=True, ch_names=ch_names, 
                                save=False, save_path='', save_file='', 
                                xy_pixel=1, z_pixel=1, ddtype='uint16')

compiling_files:  10%|█         | 1/10 [00:20<03:04, 20.55s/it]

split_convert runtime 20.408579634999995


compiling_files:  20%|██        | 2/10 [00:40<02:43, 20.42s/it]

split_convert runtime 20.213132924999996


compiling_files:  30%|███       | 3/10 [01:01<02:22, 20.37s/it]

...

compiling_files:  90%|█████████ | 9/10 [02:59<00:19, 19.73s/it]

split_convert runtime 19.466358325000016


compiling_files: 100%|██████████| 10/10 [03:19<00:00, 19.94s/it]

split_convert runtime 19.41588670599998
dict_keys(['GFP', 'red']) <class 'list'> 10
compiling the GFP channel


compiling the red channel
files_to_4D runtime 200.849550234


In [10]:
# measuring the cosine_similarity check of GFP channel before registration
pre_checks = datautils.similarity_4D(image_4D[ch_names[0]], save=True, save_path=save_path, save_file='pre_checks.csv')

cosine_sim for timepoint: 100%|██████████| 9/9 [00:06<00:00,  1.44it/s]

worked
finished measuring similarity check 6.240511722000008


In [17]:
# running antspy, saving output into temp 3D files, and then compiling them to 4D and finally deleting the temp 3D files
for method in ants_method:
    start_time = timer()
    name_ch1 = method+'_'+ch_names[0]+'_'
    name_ch2 = method+'_'+ch_names[1]+'_'
    name_shifts = method+csv_name
    antspy_drift_corr(img_4D_g=image_4D[ch_names[0]],
                      img_4D_r=image_4D[ch_names[1]], 
                      ch_names=ch_names,
                      save_path=save_path, 
                      save_name=save_name,
                      ref_t = int(len(image_4D[ch_names[0]])/2),
                      name_shifts=name_shifts,
                      drift_corr=method)
    
    gfp_list = datautils.get_file_names(path=save_path, group_by=ch_names[0]+save_name, order=False, 
                                nested_files=False, criteria='tif')
    gfp_4D = datautils.files_to_4D(files_list=gfp_list, ch_names=[ch_names[0]])
    gfp_name = method+'_4D_'+ch_names[0]+'_'+group_by+'.tif'
    datautils.save_image(gfp_name, gfp_4D, xy_pixel=0.0764616, z_pixel=0.4)
    for file in gfp_list:
        os.remove(file)

    red_list = datautils.get_file_names(path=save_path, group_by=ch_names[1]+save_name, order=False, 
                                nested_files=False, criteria='tif')
    red_4D = datautils.files_to_4D(files_list=red_list, ch_names=[ch_names[1]])
    red_name = method+'_4D_'+ch_names[1]+'_'+group_by+'.tif'
    datautils.save_image(red_name, red_4D, xy_pixel=0.0764616, z_pixel=0.4)
    for file in red_list:
        os.remove(file)
    
    # measuring the cosine_similarity check of GFP channel after registration
    post_checks = datautils.similarity_4D(gfp_4D, save=True, 
                                            save_path=save_path, 
                                            save_file=method+'_checks.csv')

    # deleting the final compiled 4D images
    del gfp_4D, red_4D 

    print(method, ' is done, and it took', timer()-start_time)   

  0%|          | 0/11 [00:00<?, ?it/s]


AttributeError: module 'ants' has no attribute 'from_numpy'